In [83]:
# pip install openbb

In [84]:
from openbb import obb
import pandas as pd

start_date = "2017-01-01"

# =============================
# 1. BTC-USD & ETH-USD
# =============================
btc = obb.crypto.price.historical(
    symbol="BTC-USD",
    start_date=start_date,
    interval="1d",
).to_df()

'''
eth = obb.crypto.price.historical(
    symbol="ETH-USD",
    start_date=start_date,
    interval="1d",
).to_df()
'''

print("BTC columns:", btc.columns)
#print("ETH columns:", eth.columns)

# Add prefix
#eth_prefixed = eth.add_prefix("ETH_")   


# ==========================================
# SP500 / GOLD / DXY
# ==========================================

# 2.1 S&P 500 
sp500 = obb.equity.price.historical(
    symbol="^GSPC",
    provider="yfinance",
    start_date=start_date,
    interval="1d",
).to_df()

# 2.2 GOLD
gold = obb.equity.price.historical(
    symbol="GC=F",
    provider="yfinance",
    start_date=start_date,
    interval="1d",
).to_df()

# 2.3 DXY
dxy = obb.equity.price.historical(
    symbol="DX-Y.NYB",
    provider="yfinance",
    start_date=start_date,
    interval="1d",
).to_df()

print("SP500 columns:", sp500.columns)
print("GOLD columns:", gold.columns)
print("DXY columns:", dxy.columns)

# Only take the close price
sp500_close = sp500["close"].rename("SP500")
gold_close  = gold["close"].rename("GOLD")
dxy_close   = dxy["close"].rename("DXY")


# ==========================================
# Merge the final table
# ==========================================

all_data = pd.concat(
    [btc, sp500_close, gold_close, dxy_close],
    axis=1
)  

print(all_data.head())

# To CSV
all_data.to_csv("BTC_raw.csv")


BTC columns: Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')
SP500 columns: Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')
GOLD columns: Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')
DXY columns: Index(['open', 'high', 'low', 'close', 'volume'], dtype='object')
                   open         high          low        close       volume  \
date                                                                          
2017-01-01   963.658020  1003.080017   958.698975   998.325012  147775008.0   
2017-01-02   998.617004  1031.390015   996.702026  1021.750000  222184992.0   
2017-01-03  1021.599976  1044.079956  1021.599976  1043.839966  185168000.0   
2017-01-04  1044.400024  1159.420044  1044.400024  1154.729980  344945984.0   
2017-01-05  1156.729980  1191.099976   910.416992  1013.380005  510199008.0   

                  SP500         GOLD         DXY  
date                                              
2017-01-01          NaN  

In [85]:
import requests
import pandas as pd

def get_cc_eth_2017():
    """
    从 CryptoCompare 下载 2017 年 ETH-USD 日度 OHLCV
    """
    url = "https://min-api.cryptocompare.com/data/v2/histoday"
    # toTs 设成 2017-12-31 23:59:59
    to_ts = int(pd.Timestamp("2017-12-31", tz="UTC").timestamp())

    params = {
        "fsym": "ETH",
        "tsym": "USD",
        "limit": 364,   # 返回最近 limit+1 天，这里足够覆盖 2017 一整年
        "toTs": to_ts
    }

    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()

    if data["Response"] != "Success":
        raise RuntimeError(f"CryptoCompare error: {data}")

    rows = data["Data"]["Data"]
    df = pd.DataFrame(rows)

    # time 是秒级时间戳
    df["date"] = pd.to_datetime(df["time"], unit="s", utc=True).dt.strftime("%Y-%m-%d")
    df.set_index("date", inplace=True)

    # CryptoCompare 的字段：
    # open, high, low, close, volumefrom (币数量), volumeto (以 USD 计价的成交额)
    df = df.rename(columns={
        "volumefrom": "volume_base",
        "volumeto": "volume_quote"
    })

    # 你如果只想一个 volume，可以选一个，比如 base 量：
    df["volume"] = df["volume_base"]

    return df[["open", "high", "low", "close", "volume"]]

eth_2017 = get_cc_eth_2017()

import yfinance as yf
eth_yf = yf.download("ETH-USD",start="2018-01-01",multi_level_index=False)
eth_yf.index = eth_yf.index.strftime("%Y-%m-%d")
eth_yf = eth_yf.rename(columns=str.lower)

# 避免重叠日期重复，这里假设 2018 之后完全用 yfinance 数据
combined = pd.concat([eth_2017, eth_yf], axis=0).sort_index()
combined.index.name = "date"




[*********************100%***********************]  1 of 1 completed


In [86]:

# ==========================================
# Merge the final table
# ==========================================

print(sp500.index)
sp500_close.index = pd.to_datetime(sp500_close.index)
gold_close.index = pd.to_datetime(gold_close.index)
dxy_close.index = pd.to_datetime(dxy_close.index)
combined.index = pd.to_datetime(combined.index)

print(combined.index)

all_data = pd.concat(
    [combined, sp500_close, gold_close, dxy_close],
    axis = 1
)  

print(all_data.head())


# To CSV
all_data.to_csv("ETH_raw.csv")

Index([2017-01-03, 2017-01-04, 2017-01-05, 2017-01-06, 2017-01-09, 2017-01-10,
       2017-01-11, 2017-01-12, 2017-01-13, 2017-01-17,
       ...
       2025-11-13, 2025-11-14, 2025-11-17, 2025-11-18, 2025-11-19, 2025-11-20,
       2025-11-21, 2025-11-24, 2025-11-25, 2025-11-26],
      dtype='object', name='date', length=2239)
DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2025-11-17', '2025-11-18', '2025-11-19', '2025-11-20',
               '2025-11-21', '2025-11-22', '2025-11-23', '2025-11-24',
               '2025-11-25', '2025-11-27'],
              dtype='datetime64[ns]', name='date', length=3252, freq=None)
              open    high     low   close      volume        SP500  \
date                                                                  
2017-01-01   8.018   8.891   7.970   8.154   854000.05          N